# Age

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from catboost import CatBoostRegressor, Pool

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
le = LabelEncoder()
scaler = StandardScaler()

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv("../../data/titanic/prepare.csv").set_index("PassengerId")
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Title,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries,IsTest,CabinSeries,isCabinSeriesPreds,isAgePreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,-0.503595,NaN,...,Mr,2,0,0,100,2,0,5,1,0
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,0.734503,C85,...,Mrs,2,0,12,182,3,0,2,0,0
3,1.0,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,-0.490544,NaN,...,Miss,1,1,21,329,1,0,4,1,0
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,0.382925,C123,...,Mrs,2,0,26,267,3,0,2,0,0
5,0.0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,-0.488127,NaN,...,Mr,1,1,26,15,2,0,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",1,31.363012,0,0,A.5. 3236,-0.488127,NaN,...,Mr,1,1,2,753,2,1,5,1,1
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.000000,0,0,PC 17758,1.461829,C105,...,Other,1,1,12,593,4,1,2,0,0
1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.500000,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,Mr,1,1,19,699,2,1,4,1,0


<IPython.core.display.Javascript object>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    float64
 1   Pclass              1309 non-null   int64  
 2   Name                1309 non-null   object 
 3   Sex                 1309 non-null   int64  
 4   Age                 1309 non-null   float64
 5   SibSp               1309 non-null   int64  
 6   Parch               1309 non-null   int64  
 7   Ticket              1309 non-null   object 
 8   Fare                1309 non-null   float64
 9   Cabin               295 non-null    object 
 10  Embarked            1309 non-null   int64  
 11  Surname             1309 non-null   object 
 12  HasNamesakes        1309 non-null   int64  
 13  Title               1309 non-null   object 
 14  FamilySize          1309 non-null   int64  
 15  IsAlone             1309 non-null   int64  
 16  Ticket

<IPython.core.display.Javascript object>

In [6]:
X = df[
    [
        "Pclass",
        "Sex",
        "SibSp",
        "Parch",
        "Fare",
        "Embarked",
        "HasNamesakes",
        "FamilySize",
        "IsAlone",
        "TicketSeries",
        "SurnameSeries",
        "TitleSeries",
    ]
]
y = df[["Age"]]
X

,Pclass,Sex,SibSp,Parch,Fare,Embarked,HasNamesakes,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries
PassengerId,,,,,,,,,,,,
1,3,1,1,0,-0.503595,2,0,2,0,0,100,2
2,1,0,1,0,0.734503,0,0,2,0,12,182,3
3,3,0,0,0,-0.490544,2,0,1,1,21,329,1
4,1,0,1,0,0.382925,2,0,2,0,26,267,3
5,3,1,0,0,-0.488127,2,0,1,1,26,15,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,0,0,-0.488127,2,0,1,1,2,753,2
1306,1,0,0,0,1.461829,0,0,1,1,12,593,4
1307,3,1,0,0,-0.503595,2,0,1,1,19,699,2


<IPython.core.display.Javascript object>

In [7]:
X.columns

Index(['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'HasNamesakes',
       'FamilySize', 'IsAlone', 'TicketSeries', 'SurnameSeries',
       'TitleSeries'],
      dtype='object')

<IPython.core.display.Javascript object>

In [8]:
y["Age"].isna().value_counts()

False    1309
Name: Age, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
train_index = y[~y["Age"].isna()].index
train_index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', name='PassengerId', length=1309)

<IPython.core.display.Javascript object>

In [10]:
X_train = X[X.index.isin(train_index)]
X_test = X[~X.index.isin(train_index)]

y_train = y[y.index.isin(train_index)]

X_train.shape, X_test.shape, y_train.shape

((1309, 12), (0, 12), (1309, 1))

<IPython.core.display.Javascript object>

In [11]:
y_train.describe()

,Age
count,1309.000000
mean,29.914179
std,13.495097
min,0.170000
25%,22.000000
50%,29.000000
75%,37.425100
max,80.000000


<IPython.core.display.Javascript object>

# Train

In [12]:
model = CatBoostRegressor()

<IPython.core.display.Javascript object>

In [13]:
model.fit(Pool(X_train, y_train), verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

<IPython.core.display.Javascript object>

# Save

In [14]:
y_preds = model.predict(X_test)
y_preds

No objects info loaded


array([], dtype=float64)

<IPython.core.display.Javascript object>

In [15]:
y_test = pd.DataFrame({"PassengerId": X_test.index, "Age": y_preds}).set_index(
    "PassengerId"
)
y_test

,Age
PassengerId,


<IPython.core.display.Javascript object>

In [16]:
df["isAgePreds"] = df["Age"].isna().astype(int)

<IPython.core.display.Javascript object>

In [17]:
df.fillna(y_test, inplace=True)

<IPython.core.display.Javascript object>

In [18]:
# normalize
df["Age"] = scaler.fit_transform(df[["Age"]])

<IPython.core.display.Javascript object>

In [19]:
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Title,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries,IsTest,CabinSeries,isCabinSeriesPreds,isAgePreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,-0.586673,1,0,A/5 21171,-0.503595,NaN,...,Mr,2,0,0,100,2,0,5,1,0
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.599396,1,0,PC 17599,0.734503,C85,...,Mrs,2,0,12,182,3,0,2,0,0
3,1.0,3,"Heikkinen, Miss. Laina",0,-0.290155,0,0,STON/O2. 3101282,-0.490544,NaN,...,Miss,1,1,21,329,1,0,4,1,0
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.377008,1,0,113803,0.382925,C123,...,Mrs,2,0,26,267,3,0,2,0,0
5,0.0,3,"Allen, Mr. William Henry",1,0.377008,0,0,373450,-0.488127,NaN,...,Mr,1,1,26,15,2,0,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",1,0.107401,0,0,A.5. 3236,-0.488127,NaN,...,Mr,1,1,2,753,2,1,5,1,0
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,0.673526,0,0,PC 17758,1.461829,C105,...,Other,1,1,12,593,4,1,2,0,0
1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,0.636461,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,Mr,1,1,19,699,2,1,4,1,0


<IPython.core.display.Javascript object>

In [20]:
df.to_csv("../../data/titanic/prepare.csv")

<IPython.core.display.Javascript object>